# Import Statements

In [85]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
import pandas as pd 
import os 

# Create the RFCModel class which holds the model, dataframe, and the teams of each league with their respective winrates  

In [86]:
class RFCModel():
    def __init__(self, league):
        self.clf = None 
        self.model_accuracy = None 
        self.league = league
        self.teams = {}
        self.df = None 
        self.df_no_teams = None 
        self.sorted_keys = None 
        
    def calculate_rankings(self):
        
        teams = self.df['team'].unique()
        for team in teams:
            # team_all_matches = pd.DataFrame(columns=self.df_no_teams.columns)
            # team_all_matches.columns = team_all_matches.columns.astype(str) 
            wins, losses = 0, 0
            
            for index, row in self.df.iterrows():
                if row['team'] == team:
                    new_row = (self.df_no_teams.iloc[index].drop(labels=['result']).values.reshape(1,-1))
                    predictions = self.clf.predict(new_row)
                    if predictions:
                        wins += 1 
                    else:
                        losses += 1 
            self.teams[team] = wins / (wins + losses) * 100 

        
    def create_model(self):
        processed_data_directory = './data_by_league'
        file_name = self.league + ".csv"
        file_path  = os.path.join(processed_data_directory, file_name)
        self.df = pd.read_csv(file_path)
        self.df.dropna(inplace=True, ignore_index=True)
        self.df_no_teams = self.df.drop(columns=['team', 'elder_dragon_kills', 'game_length', 'second_turret_kill', 'third_turret_kill', 'first_blood', 'file_name'], axis=1)
        
        x = self.df_no_teams.drop(columns=['result'])
        y = self.df_no_teams['result']
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        # print(f"training data: {X_train}")
        
        # Create a Random Forest Classifier
        self.clf = RandomForestClassifier(random_state=42)
        
        # Train the model
        self.clf.fit(X_train, y_train)
        
        # Make predictions on the test set
        y_pred = self.clf.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        
        self.model_accuracy = accuracy

    def sort_keys(self):
        self.sorted_keys = sorted(self.teams, key=lambda x: self.teams[x], reverse=True)
        

# Code to assign points to each team for ranking 

In [87]:
def assign_points(teams_dict, sorted_keys, league): 
    returning_dict = {}
    curr_score = 10 
    
    # Arbitrary additional scores based on perceived strength of the entire region 
    if league == 'LPL':
        curr_score += 4 
    elif league == 'LCK':
        curr_score += 3
    elif league == 'LEC':
        curr_score += 2 
    elif league == 'LCS':
        curr_score += 1 
    
    
    prev_win_rate = teams_dict[list(teams_dict.keys())[0]] 
    for team in sorted_keys:
        win_rate = teams_dict[team]        
        if prev_win_rate == win_rate:
            returning_dict[team] = curr_score

        else:
            prev_win_rate = win_rate 
            curr_score -= 1     
            returning_dict[team] = curr_score
            
    return returning_dict

In [88]:
lck = RFCModel("LCK")


# Calculate the points for all teams  

In [ ]:
all_team_scores = {}
leagues = ["LCS", "LEC", "CBLOL", "LCK", "LJL", "LLA", "LPL", "PCS", "VCS"]

for league in leagues:
    league_model = RFCModel(league)
    league_model.create_model()
    league_model.calculate_rankings()
    league_model.sort_keys()
    
    league_dict = assign_points(league_model.teams, league_model.sorted_keys, league)
    
    all_team_scores.update(league_dict)
    
all_team_scores


/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/yslee/Desktop/LoLHackathon/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomFore

# The final list of teams, ranked best to worst 

In [ ]:
team_ranking = sorted(all_team_scores, key=lambda x: all_team_scores[x], reverse=True)
team_ranking